## File Encryption/Decryption using AES

This program utilizes the python hazmat cryptography library to encrypt and decrypt user files. The file will first be padded using PKCS7 to make sure the input to our encryption algorithm is the appropriate block size. CBC mode (AES) will then be used to encrypt a file of the user's choice and output a JSON file.

In [11]:
# import the necessary libraries needed
import os as os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import json

In [2]:
# constants used for our encryption algorithm
IV_LENGTH = 16
KEY_LENGTH = 32
BLOCK_SIZE = 128

In [3]:
def myEncrypt(message):
    """
    Encrypts the message using CBC mode (AES).
    
    Arguments:
    message -- stream of bytes the user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    """
    
    # generate our key and iv
    backend = default_backend()
    iv = os.urandom(IV_LENGTH)
    key = os.urandom(KEY_LENGTH)
    
    # pad to approprate block size
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    paddedMessage = padder.update(message)
    paddedMessage += padder.finalize()
    
    # encrypt using CBC mode (AES)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    encryptor = cipher.encryptor()
    ct = encryptor.update(paddedMessage) + encryptor.finalize()
    
    return ct, iv, key

In [4]:
# test case for a string
message = "A Secret Message for Mehrdad Aliasgari the great"
byteMessage = message.encode('utf-8')
print(type(byteMessage))
print(byteMessage)
testMsg, testIV, testKey = myEncrypt(byteMessage)

<class 'bytes'>
b'A Secret Message for Mehrdad Aliasgari the great'


In [5]:
def myFileEncrypt(filepath):
    """
    Takes the file from the filepath specified by the user and encrypts the file. 
    
    Arguments:
    filepath -- Filepath for the file user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    fileType -- the file extension
    """
    # read the file and convert it to bytes
    file = open(filepath, "rb")
    byte = bytes(file.read())
    file.close()
    
    # encrypt the file
    cipher, iv, key = myEncrypt(byte)
    fileType = "PNG"
    return cipher, iv, key, fileType

In [26]:
cipher, iv, key, fileType = myFileEncrypt("test.png")
files = {
    "cipher" : str(cipher),
    "iv" : str(iv),
    "key": str(key),
    "ext"  : fileType   
}
fileName = open("encrypt.json", "w")
jsonObject = json.dump(files,fileName)

file = open("out.bytes", "wb")
file.write(cipher)
file.close()

In [8]:
file2 = open("out.bytes", "rb")
byte2 = bytes(file2.read())


In [9]:
def myDecrypt(message, key, iv):
    """
    Decrypts the message using CBC mode (AES).
    
    Arguments:
    message -- stream of bytes the user wants to encrypt
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    
    Return:
    unpaddedMessage -- the original message
    """
    
    # decrypt the file
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    decryptedMessage = decryptor.update(message) + decryptor.finalize()
    
    # unpad the file
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    unpaddedMessage = unpadder.update(decryptedMessage)
    
    return unpaddedMessage

In [10]:
def myFileDecrypt(filepath, iv, key, fileType):
    """
    Takes the file from the filepath specified by the user and encrypts the file. 
    
    Arguments:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    fileType -- the file extension
    """    
    file = open(filepath, "rb")
    byte = bytes(file.read())
    decryptedFile = myDecrypt(byte, key, iv)
    image = Image.open(io.BytesIO(decryptedFile))
    image.show()

In [12]:
myFileDecrypt("out.bytes",iv, key, "PNG")

C:\Users\piercingonez\Anaconda3\lib\site-packages\PIL\Image.py:888: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
